# Linked Data Representations and Nested Data Structures

One of the most surprising things about programming can be how
many different ways there are to code up the same informal idea. The
space of choices can usually be divided between data structures, or
the ways of storing data, and algorithms, or the ways of working with
the data structures to compute certain results. In this lecture, we
explore a variety of different implementations of a common high-level
data model and some operations on it. In the process, we learn
some general techniques for encoding data, including via multiple
levels of data structures nested inside of other structures.

Our motto throughout will be that choosing a data structure is a careful
balancing between concerns of **how easy the code is to write** and **how
well the code performs**. Performance can be broken out into
dimensions like speed and memory usage, but today we will focus
on speed.  We will also consider how easy it is to store the data representation on disk in a database, instead of just in memory within a Python program.

## Preface: What is an Object?

One of the most prevalent terms in programming is **object**. We've learned
about object-oriented programming in previous lectures. In this lecture,
we'll avoid Python's explicit object-orientation features to show that the
idea of an object is more fundamental.

For this lecture, an object is any value with a small fixed set of
**fields** holding further values. For instance, an object representing a
student might contain a field for the student ID (a positive integer) and a
field for the student's name (a string of text). An object standing
for an academic major might include a course number (a positive
integer), a name (a string of text), and a roster of participating students
(a list of student objects). This concept meshes nicely with
informal ways of describing all sorts of real-world ideas.

Python `class`es can definitely do this, and are a good choice.  But another, simpler way to represent an object like this is with a Python `dict` with a small fixed set of keys.  For example:

In [2]:
student1 = {"id": 90001, "name": "Alice"}
student2 = {"id": 90002, "name": "Bob"}
major1 = {"num": 42, "name": "Blah Blah Studies",
          "students": [student1, student2]}
print(student1["name"])
print(" and ".join(student["name"] for student in major1["students"]))

Alice
Alice and Bob


## JSON representation of objects

We've seen this representation in several labs already.  Representing objects by `dict`s has become particularly popular lately with the rise of [JSON (JavaScript Object Notation)](http://www.json.org/) as a simple way to represent objects in files, databases, and on the web.  Specifically, JSON is a subset of Python (and a subset of JavaScript) that can express numbers, strings, arrays (`list`s), and dictionaries (`dict`s) whose keys are all strings, provided the array elements and dictionary values are all themselves numbers, strings, arrays, or dictionaries, recursively.  In particular, JSON can represent objects like those above, as well as nested objects where the field values are themselves objects (recursively).

Python has a built-in module `json` for outputting Python objects into JSON, and (more importantly) safely parsing JSON (which is rather strict) into Python objects.  The notation is extremely familiar, and you could also get away with the built-in functions `repr` and `eval` instead, but the latter is **unsafe** in case you don't trust the input string you're parsing.

In [3]:
import json
j = json.dumps(major1)
j

'{"num": 42, "name": "Blah Blah Studies", "students": [{"id": 90001, "name": "Alice"}, {"id": 90002, "name": "Bob"}]}'

In [4]:
r = repr(major1)
r

"{'num': 42, 'name': 'Blah Blah Studies', 'students': [{'id': 90001, 'name': 'Alice'}, {'id': 90002, 'name': 'Bob'}]}"

In [5]:
j == r.replace("'", '"')

True

In [5]:
json.loads(j)

{'num': 42,
 'name': 'Blah Blah Studies',
 'students': [{'id': 90001, 'name': 'Alice'}, {'id': 90002, 'name': 'Bob'}]}

In [6]:
eval(j) # don't do this unless you trust j!

{'num': 42,
 'name': 'Blah Blah Studies',
 'students': [{'id': 90001, 'name': 'Alice'}, {'id': 90002, 'name': 'Bob'}]}

### JSON in web APIs

JSON is useful because it's simple (so it's easy to parse), flexible (supports nesting), and works in most programming languages.  As a result, it's usually the output format (and sometimes the input) for most web APIs.  As a random example, [here's an API](https://dog.ceo/dog-api/) that returns JSON:

In [9]:
from urllib.request import urlopen
data = urlopen("https://dog.ceo/api/breeds/image/random").read()
data

b'{"message":"https:\\/\\/images.dog.ceo\\/breeds\\/rottweiler\\/n02106550_7473.jpg","status":"success"}'

In [10]:
url = json.loads(data)["message"]
url

'https://images.dog.ceo/breeds/rottweiler/n02106550_7473.jpg'

In [11]:
from IPython.display import Image
Image(url=url)

There's even an entire database system, [MongoDB](https://www.mongodb.com/), that supports storing a bunch of JSON "documents" and searching among them, with indexing (like we'll see below) and lots of other fancy database features.  So being able to represent our objects in JSON form is important if we want to use this database system.  Most other databases, like [MySQL](https://www.mysql.com/) or [PostgreSQL](https://www.postgresql.org/), are **relational**, which means they require data objects to be in an even *more* stringent format: most often, objects need to be "flat", meaning that the value in each field is a basic type (number or string) instead of a recursive list or dictionary.

## Running Example

Let's consider the following scenario, motivated by [the web system](https://catsoop.org/website) that runs 6.009.  We want to represent the following interrelated pieces of data:

* **Students**, each with a name and kerberos
* **Psets**, each with a number and maximum number of points
* **Grades**, each assigning a number of points (score) to one student for a particular pset

One aspect that makes this scenario interesting is that a grade needs to be able to refer to a student and a pset.  As a result, we call this **linked data**.  This is closely related to the idea of a linked data structure, which we saw with linked lists and binary search trees — but that form of linking was an *implementation detail* for how to represent a set of items, whereas here our *goal* is to store data with links.

There are many different ways to represent this kind of linked data.

## Representation 1: Linked Data with Linked Data Structures

Perhaps the most natural way to represent linked data is with actual links, i.e., references/pointers to the relevant objects, like we did above.  For example:

In [10]:
student1 = {"name": "Alice", "kerberos": "al1ce"}
student2 = {"name": "Bob", "kerberos": "b0b"}
pset1 = {"number": 1, "points": 10}
pset2 = {"number": 2, "points": 15}
grade1 = {"pset": pset1, "student": student1, "points": 9.5}
grade2 = {"pset": pset1, "student": student2, "points": 6}
grade3 = {"pset": pset2, "student": student2, "points": 12}
[grade1, grade2, grade3]

[{'pset': {'number': 1, 'points': 10},
  'student': {'name': 'Alice', 'kerberos': 'al1ce'},
  'points': 9.5},
 {'pset': {'number': 1, 'points': 10},
  'student': {'name': 'Bob', 'kerberos': 'b0b'},
  'points': 6},
 {'pset': {'number': 2, 'points': 15},
  'student': {'name': 'Bob', 'kerberos': 'b0b'},
  'points': 12}]

### Aliasing

Note that `grade1` and `grade2` link to the *same* `pset1` object, and `grade2` and `grade3` link to the *same* `student2` object.  This is called **aliasing**.  In past labs, you've had to take care that aliasing didn't hurt you.  Here, it's more of a feature: a result of the aliasing is that it's possible to change a feature of one of the linked objects, and all the relevant grade objects will effectively update.  This is great for fixing mistakes, but also must be used with care: aliased objects can also have unintended effects where you expect to update only one thing but update everything.

In [11]:
student2["name"] = "Robert"    # changes student2 for everyone
grade1["pset"]["points"] = 11  # changes pset1 for everyone
[grade1, grade2, grade3]

[{'pset': {'number': 1, 'points': 11},
  'student': {'name': 'Alice', 'kerberos': 'al1ce'},
  'points': 9.5},
 {'pset': {'number': 1, 'points': 11},
  'student': {'name': 'Robert', 'kerberos': 'b0b'},
  'points': 6},
 {'pset': {'number': 2, 'points': 15},
  'student': {'name': 'Robert', 'kerberos': 'b0b'},
  'points': 12}]

### Interface and Implementation

Now let's turn this idea into an implementation of an interface (ADT) so that we can explore other implementations.  Notably, in addition to actually building the relevant objects, we need to store them somehow.  Let's just keep a list of all objects of each type.

In [12]:
class LinkedRep:
    def __init__(self):
        self.students = []
        self.psets = []
        self.grades = []
    def addStudent(self, name, kerberos):
        student = {"name": name, "kerberos": kerberos}
        self.students.append(student)
        return student
    def addPset(self, number, points):
        pset = {"number": number, "points": points}
        self.psets.append(pset)
        return pset
    def addGrade(self, pset, student, points):
        grade = {"pset": pset, "student": student, "points": points}
        self.grades.append(grade)
        return grade

In [13]:
r = LinkedRep()
r.addStudent("Alice", "al1ce")
r.addStudent("Bob", "b0b")
r.addPset(1, 10)
r.addPset(2, 15)
#r.addGrade(pset with number 1, student with kerberos "al1ce", 9.5)??

{'number': 2, 'points': 15}

So far so good, but to add a grade, we need to actually be able to look up psets and students.  Let's add some methods to do that:

In [14]:
class LinkedRep(LinkedRep):
    def studentsWithName(self, name): # don't assume unique
        return [student for student in self.students
                if student["name"] == name]
    def studentWithKerberos(self, kerberos): # assume unique
        return [student for student in self.students
                if student["kerberos"] == kerberos][0]
    def psetWithNumber(self, number): # assume unique
        return [pset for pset in self.psets
                if pset["number"] == number][0]

In [15]:
r = LinkedRep()
r.addStudent("Alice", "al1ce")
r.addStudent("Bob", "b0b")
r.addPset(1, 10)
r.addPset(2, 15)
r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("al1ce"), 9.5)
r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("b0b"), 6)
r.addGrade(r.psetWithNumber(2), r.studentWithKerberos("b0b"), 12)
r.grades

[{'pset': {'number': 1, 'points': 10},
  'student': {'name': 'Alice', 'kerberos': 'al1ce'},
  'points': 9.5},
 {'pset': {'number': 1, 'points': 10},
  'student': {'name': 'Bob', 'kerberos': 'b0b'},
  'points': 6},
 {'pset': {'number': 2, 'points': 15},
  'student': {'name': 'Bob', 'kerberos': 'b0b'},
  'points': 12}]

Now we have a clean interface that produces the linked objects we have in mind.  This code isn't particularly efficient yet: every search like `psetWithNumber` requires a linear scan over the list of *all* psets.  Before we make things more efficient, let's consider other representations we could use.  Let's also define a function for the example above so we don't need to keep repeating the test code.

In [16]:
def example(Rep):
    r = Rep()
    r.addStudent("Alice", "al1ce")
    r.addStudent("Bob", "b0b")
    r.addPset(1, 10)
    r.addPset(2, 15)
    r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("al1ce"), 9.5)
    r.addGrade(r.psetWithNumber(1), r.studentWithKerberos("b0b"), 6)
    r.addGrade(r.psetWithNumber(2), r.studentWithKerberos("b0b"), 12)
    return r

## Representation 2: Linked Data with Unique IDs

### Serialization

One challenge with the links in Representation 1 is that it's hard to store objects on disk.  Files on disk can only store binary strings, not objects with links.  What we're looking for is  a **serialization** scheme for converting objects into binary strings.  JSON is one natural choice, but unfortunately this won't handle our case well because it doesn't notice the repeated objects.  (Even worse, if we had cycles in our links, the conversion would fail altogether.)

In [17]:
json.dumps([r.students,r.psets,r.grades])

'[[{"name": "Alice", "kerberos": "al1ce"}, {"name": "Bob", "kerberos": "b0b"}], [{"number": 1, "points": 10}, {"number": 2, "points": 15}], [{"pset": {"number": 1, "points": 10}, "student": {"name": "Alice", "kerberos": "al1ce"}, "points": 9.5}, {"pset": {"number": 1, "points": 10}, "student": {"name": "Bob", "kerberos": "b0b"}, "points": 6}, {"pset": {"number": 2, "points": 15}, "student": {"name": "Bob", "kerberos": "b0b"}, "points": 12}]]'

Python offers the `pickle` library to serialize nested data structures, even with cycles, into a binary string that you could save to a file. Notice that the result has only one instance of "Alice", one instance of "Bob", etc.

In [18]:
import pickle
pickle.dumps([r.students,r.psets,r.grades])

b'\x80\x03]q\x00(]q\x01(}q\x02(X\x04\x00\x00\x00nameq\x03X\x05\x00\x00\x00Aliceq\x04X\x08\x00\x00\x00kerberosq\x05X\x05\x00\x00\x00al1ceq\x06u}q\x07(h\x03X\x03\x00\x00\x00Bobq\x08h\x05X\x03\x00\x00\x00b0bq\tue]q\n(}q\x0b(X\x06\x00\x00\x00numberq\x0cK\x01X\x06\x00\x00\x00pointsq\rK\nu}q\x0e(h\x0cK\x02h\rK\x0fue]q\x0f(}q\x10(X\x04\x00\x00\x00psetq\x11h\x0bX\x07\x00\x00\x00studentq\x12h\x02h\rG@#\x00\x00\x00\x00\x00\x00u}q\x13(h\x11h\x0bh\x12h\x07h\rK\x06u}q\x14(h\x11h\x0eh\x12h\x07h\rK\x0cuee.'

This is fine if you want to store the entire database in one file, and change the entire file every time the database changes.  But for large databases, you'd like to be able to load/store/update individual objects without having to touch the entire database.  In particular, to use any off-the-shelf database system (like the aforementioned MongoDB, MySQL, or PostgreSQL), we need to be able to serialize individual objects.

### Storing IDs instead of references/pointers

A common approach to fixing this problem is to give every object a unique ID (at least, unique within the category of things it's in), and represent links with IDs instead of pointers/references.  For example:

In [19]:
student1 = {"id": 1, "name": "Alice", "kerberos": "al1ce"}
student2 = {"id": 2, "name": "Bob", "kerberos": "b0b"}
pset1 = {"id": 3, "number": 1, "points": 10}
pset2 = {"id": 4, "number": 2, "points": 15}
grade1 = {"id": 5, "pset": 3, "student": 1, "points": 9.5}
grade2 = {"id": 6, "pset": 3, "student": 2, "points": 6}
grade3 = {"id": 7, "pset": 4, "student": 2, "points": 12}
[grade1, grade2, grade3]

[{'id': 5, 'pset': 3, 'student': 1, 'points': 9.5},
 {'id': 6, 'pset': 3, 'student': 2, 'points': 6},
 {'id': 7, 'pset': 4, 'student': 2, 'points': 12}]

### Implementing the Interface

This code is ugly to write in this form, but we can make it clean again by implementing the same interface as above.  The method `addGrade` still expects `pset` and `student` to be dictionary objects, but instead of storing a reference to them, it stores their `id` field.  We also add `id` fields automatically when adding anything to the database.  The rest of the code is the same.

In [20]:
class IDRep:
    def __init__(self):
        self.students = []
        self.psets = []
        self.grades = []
        self.nextId = 1
    def addStudent(self, name, kerberos):
        student = {"id": self.nextId, "name": name, "kerberos": kerberos}
        self.students.append(student)
        self.nextId += 1
        return student
    def addPset(self, number, points):
        pset = {"id": self.nextId, "number": number, "points": points}
        self.psets.append(pset)
        self.nextId += 1
        return pset
    def addGrade(self, pset, student, points):
        grade = {"id": self.nextId, "pset": pset["id"],
                 "student": student["id"], "points": points}
        self.grades.append(grade)
        self.nextId += 1
        return grade
    def studentsWithName(self, name): # don't assume unique
        return [student for student in self.students
                if student["name"] == name]
    def studentWithKerberos(self, kerberos): # assume unique
        return [student for student in self.students
                if student["kerberos"] == kerberos][0]
    def psetWithNumber(self, number): # assume unique
        return [pset for pset in self.psets
                if pset["number"] == number][0]

In [21]:
r = example(IDRep)
r.grades

[{'id': 5, 'pset': 3, 'student': 1, 'points': 9.5},
 {'id': 6, 'pset': 3, 'student': 2, 'points': 6},
 {'id': 7, 'pset': 4, 'student': 2, 'points': 12}]

As promised, the nice thing now is that we can encode individual objects without having to encode all the objects they point to, making this representation a lot better for on-disk databases:

In [22]:
json.dumps(r.grades[0])

'{"id": 5, "pset": 3, "student": 1, "points": 9.5}'

### Working with IDs and expansion

However, if we want to actually use a grade object like this, we need to be able to look up psets and students by their ID.  We can easily write some more methods to achieve this, similar to the `...With...` methods above.  We can also make an "expand" helper to look up all the data associated with a grade — but note that this makes a new `dict` object, so you wouldn't want to modify anything in it (as it wouldn't actually change the database).

In [23]:
class IDRep(IDRep):
    def studentWithID(self, id): # assume unique
        return [student for student in self.students
                if student["id"] == id][0]
    def psetWithID(self, id): # assume unique
        return [pset for pset in self.psets
                if pset["id"] == id][0]
    # for good measure:
    def gradeWithID(self, id): # assume unique
        return [grade for grade in self.grades
                if grade["id"] == id][0]
    def expandGrade(self, grade):
        return dict(grade,
            pset = self.psetWithID(grade["pset"]),
            student = self.studentWithID(grade["student"]))

In [24]:
r = example(IDRep)
g1 = r.grades[0]
g1

{'id': 5, 'pset': 3, 'student': 1, 'points': 9.5}

In [25]:
r.studentWithID(g1["student"])

{'id': 1, 'name': 'Alice', 'kerberos': 'al1ce'}

In [26]:
e1 = r.expandGrade(g1)
e1

{'id': 5,
 'pset': {'id': 3, 'number': 1, 'points': 10},
 'student': {'id': 1, 'name': 'Alice', 'kerberos': 'al1ce'},
 'points': 9.5}

In [27]:
e1["points"] = 10  # shouldn't do this: updates the expanded copy, not the database

In [28]:
e1  # expanded copy

{'id': 5,
 'pset': {'id': 3, 'number': 1, 'points': 10},
 'student': {'id': 1, 'name': 'Alice', 'kerberos': 'al1ce'},
 'points': 10}

In [29]:
g1  # database

{'id': 5, 'pset': 3, 'student': 1, 'points': 9.5}

## Speeding Up with Dictionary Indexes

Now it's time to speed up our database.  In both representations (`LinkedRep` and `IDRep`), we currently need a `for` loop to search for matching students/psets/grades.  How can we speed this up?  By adding a `dict` that maps each query of interest to the matching object(s)!  In the database world, these are called [**indexes**](https://en.wikipedia.org/wiki/Database_index).  An index lets us implement the same query interface by just looking it up in the dictionary.  Of course, for this to work, we also need to modify the `add...` methods to update the relevant dictionaries, which incurs some overhead.  In real life, you might only want to add an index for query types that are frequent enough, and only when the database is large enough, to warrent it.

### Indexes in Linked Representation (1)

In [30]:
class IndexedLinkedRep:
    def __init__(self):
        self.students = []
        self.psets = []
        self.grades = []
        self.studentsByName = {}                                 #NEW
        self.studentByKerberos = {}                              #NEW
        self.psetByNumber = {}                                   #NEW
    def addStudent(self, name, kerberos):
        assert kerberos not in self.studentByKerberos # need unique
        student = {"name": name, "kerberos": kerberos}
        self.students.append(student)
        self.studentsByName.setdefault(name, []).append(student) #NEW
        self.studentByKerberos[kerberos] = student               #NEW
        return student
    def addPset(self, number, points):
        assert number not in self.psetByNumber # need unique
        pset = {"number": number, "points": points}
        self.psets.append(pset)
        self.psetByNumber[number] = pset                         #NEW
        return pset
    def addGrade(self, pset, student, points):
        grade = {"pset": pset, "student": student, "points": points}
        self.grades.append(grade)
        return grade
    def studentsWithName(self, name):
        return self.studentsByName.get(name, [])                 #NEW
    def studentWithKerberos(self, kerberos):
        return self.studentByKerberos[kerberos]                  #NEW
    def psetWithNumber(self, number):
        return self.psetByNumber[number]                         #NEW

In [31]:
r = example(IndexedLinkedRep)
r.grades

[{'pset': {'number': 1, 'points': 10},
  'student': {'name': 'Alice', 'kerberos': 'al1ce'},
  'points': 9.5},
 {'pset': {'number': 1, 'points': 10},
  'student': {'name': 'Bob', 'kerberos': 'b0b'},
  'points': 6},
 {'pset': {'number': 2, 'points': 15},
  'student': {'name': 'Bob', 'kerberos': 'b0b'},
  'points': 12}]

In [32]:
r.studentsByName

{'Alice': [{'name': 'Alice', 'kerberos': 'al1ce'}],
 'Bob': [{'name': 'Bob', 'kerberos': 'b0b'}]}

In [33]:
r.studentByKerberos

{'al1ce': {'name': 'Alice', 'kerberos': 'al1ce'},
 'b0b': {'name': 'Bob', 'kerberos': 'b0b'}}

### Indexes in ID Representation (2)

The same index approach can be applied to the ID-based representation in a similar way.  Here we store pointers to actual objects, which makes sense for an in-memory index.  In a database on disk, most indexes would actually just store IDs of the objects with the specified properties.  (Something trickier needs to happen for the `...ById` dictionaries, which we won't get into here.)

In [34]:
class IndexedIDRep:
    def __init__(self):
        self.students = []
        self.psets = []
        self.grades = []
        self.nextId = 1
        self.studentById = {}                                    #NEW
        self.studentsByName = {}                                 #NEW
        self.studentByKerberos = {}                              #NEW
        self.psetById = {}                                       #NEW
        self.psetByNumber = {}                                   #NEW
        self.gradeById = {}                                      #NEW
    def addStudent(self, name, kerberos):
        assert self.nextId not in self.studentById # should be unique
        assert kerberos not in self.studentByKerberos # need unique
        student = {"id": self.nextId, "name": name, "kerberos": kerberos}
        self.students.append(student)
        self.studentById[self.nextId] = student
        self.studentsByName.setdefault(name, []).append(student) #NEW
        self.studentByKerberos[kerberos] = student               #NEW
        self.nextId += 1
        return student
    def addPset(self, number, points):
        assert self.nextId not in self.studentById # should be unique
        assert number not in self.psetByNumber # need unique
        pset = {"id": self.nextId, "number": number, "points": points}
        self.psets.append(pset)
        self.psetById[self.nextId] = pset                        #NEW
        self.psetByNumber[number] = pset                         #NEW
        self.nextId += 1
        return pset
    def addGrade(self, pset, student, points):
        assert self.nextId not in self.gradeById # should be unique
        grade = {"id": self.nextId, "pset": pset["id"],
                 "student": student["id"], "points": points}
        self.grades.append(grade)
        self.gradeById[self.nextId] = grade                      #NEW
        self.nextId += 1
        return grade
    # Next three functions are identical to IndexedLinkedRep definitions above
    def studentsWithName(self, name): # don't assume unique
        return self.studentsByName.get(name, [])                 #NEW
    def studentWithKerberos(self, kerberos): # assume unique
        return self.studentByKerberos[kerberos]                  #NEW
    def psetWithNumber(self, number): # assume unique
        return self.psetByNumber[number]                         #NEW
    def studentWithID(self, id): # assume unique
        return self.studentById[id]                              #NEW
    def psetWithID(self, id): # assume unique
        return self.psetById[id]                                 #NEW
    def gradeWithID(self, id): # assume unique
        return self.gradeById[id]                                #NEW
    def expandGrade(self, grade): # as before
        return dict(grade,
            pset = self.psetWithID(grade["pset"]),
            student = self.studentWithID(grade["student"]))

In [35]:
r = example(IndexedIDRep)
r.grades[0]

{'id': 5, 'pset': 3, 'student': 1, 'points': 9.5}

In this representation, we should arguably remove the lists `students`, `psets`, `grades`, and instead use the values of the ID dictionaries `studentById`, `psetById`, `gradeById` if we ever need to iterate over everything.

### Index Maintenance

One challenge with indexing is that we need to actually *maintain* the indexes as objects change their fields.  For example, if Bob changes his name, we will no longer be able to find him through the index.

In [36]:
r.studentsWithName("Bob")

[{'id': 2, 'name': 'Bob', 'kerberos': 'b0b'}]

In [37]:
r.studentWithKerberos("b0b")["name"] = "Robert"

In [38]:
r.studentsWithName("Bob")

[{'id': 2, 'name': 'Robert', 'kerberos': 'b0b'}]

In [39]:
r.studentsWithName("Robert")

[]

The original (slow) representations `LinkedRep` and `IDRep` wouldn't have had this problem, because they look the entire list of students every time.

In [40]:
r = example(IDRep)
r.studentWithKerberos("b0b")["name"] = "Robert"
r.studentsWithName("Robert")

[{'id': 2, 'name': 'Robert', 'kerberos': 'b0b'}]

The proper way to fix this problem is to define an interface for updating objects.  For example:

In [41]:
class IndexedLinkedRep(IndexedLinkedRep):
    def changeStudentName(self, student, name):
        self.studentsByName[student["name"]].remove(student)
        student["name"] = name
        self.studentsByName.setdefault(name, []).append(student)
    def changeStudentKerberos(self, student, kerberos):
        del self.studentsByKerberos[student["kerberos"]]
        student["kerberos"] = kerberos
        self.studentsByKerberos[kerberos] = student

In [42]:
class IndexedIDRep(IndexedIDRep): # identical to above
    def changeStudentName(self, student, name):
        self.studentsByName[student["name"]].remove(student)
        student["name"] = name
        self.studentsByName.setdefault(name, []).append(student)
    def changeStudentKerberos(self, student, kerberos):
        del self.studentsByKerberos[student["kerberos"]]
        student["kerberos"] = kerberos
        self.studentsByKerberos[kerberos] = student

In [43]:
for Rep in [IndexedLinkedRep, IndexedIDRep]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    r.changeStudentName(r.studentWithKerberos("b0b"), "Robert")
    print(r.studentsWithName("Robert"))

# IndexedLinkedRep
[{'name': 'Robert', 'kerberos': 'b0b'}]
# IndexedIDRep
[{'id': 2, 'name': 'Robert', 'kerberos': 'b0b'}]


## Grade Queries

We haven't yet defined any queries on grades in our ADT.  Let's first think about what type of queries we might want to support:

1. How many points did this student get on this pset?
2. What are all the grades for this student? (for displaying a summary)
3. How many points total did this student get on all psets?
4. What are all the grades on this pset? (for plotting or computing statistics)

These are actually the first types of queries where the input is an object instead of a field value (like name, kerberos, or number).

Let's first write slow implementations of these functions, which just iterate over all the grades, to serve as specifications for how the operations should behave.

In [44]:
class LinkedRep(LinkedRep):
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        return [grade["points"] for grade in self.grades
                if grade["pset"] is pset and
                   grade["student"] is student][0]
    def gradesForStudent(self, student):
        return [grade for grade in self.grades
                if grade["student"] is student]
    def totalPointsForStudent(self, student):
        return sum(grade["points"]
                   for grade in self.gradesForStudent(student))
    def gradesForPset(self, pset):
        return [grade for grade in self.grades
                if grade["pset"] is pset]

class IDRep(IDRep):
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        return [grade["points"] for grade in self.grades
                if grade["pset"] == pset["id"] and
                   grade["student"] == student["id"]][0]
    def gradesForStudent(self, student):
        return [grade for grade in self.grades
                if grade["student"] == student["id"]]
    def totalPointsForStudent(self, student): # same as above
        return sum(grade["points"]
                   for grade in self.gradesForStudent(student))
    def gradesForPset(self, pset):
        return [grade for grade in self.grades
                if grade["pset"] == pset["id"]]

In [45]:
for Rep in [LinkedRep, IDRep]:
    print(f"# {Rep.__name__}")
    r = example(LinkedRep)
    student = r.studentWithKerberos("b0b")
    print(f"{student['name']}'s grades: {r.gradesForStudent(student)}")
    print("Total point value:", r.totalPointsForStudent(student))

# LinkedRep
Bob's grades: [{'pset': {'number': 1, 'points': 10}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 6}, {'pset': {'number': 2, 'points': 15}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 12}]
Total point value: 18
# IDRep
Bob's grades: [{'pset': {'number': 1, 'points': 10}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 6}, {'pset': {'number': 2, 'points': 15}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 12}]
Total point value: 18


Now let's turn to making these operations efficient using additional indexes.

### Student + Pset Queries

The first type of query would be supported by an index where the key consists of both a pset and a student.  We haven't yet seen an index where keys are objects.  In the ID representation, this is easy: we just make the key a tuple of pset ID and student ID.  We'll use subclassing to make clear what we need to change to support this index.

In [46]:
class IndexedIDRep1(IndexedIDRep):
    def __init__(self):
        super().__init__()
        self.gradeByPsetAndStudent = {}
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        self.gradeByPsetAndStudent[pset["id"], student["id"]] = grade
        return grade
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        return self.gradeByPsetAndStudent[pset["id"], student["id"]]["points"]

In [47]:
r = example(IndexedIDRep1)
r.pointsForPsetAndStudent(r.psetWithNumber(1),
                          r.studentWithKerberos("al1ce"))

9.5

For the linked representation, we can't use a pset-student pair as a key in the index dictionary.  (Why?  Because they're `dict`s, which are considered unhashable because of their mutability.)  But we really just want to keep track of the *identities*/*locations* of the psets and students, not their contents ⁠— that's why we used `is` above.  Python's built-in function `id` tells us where an object is located in memory, which won't change even if the contents change.

In [48]:
class IndexedLinkedRep1(IndexedLinkedRep):
    def __init__(self):
        super().__init__()
        self.gradeByPsetAndStudent = {}
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        self.gradeByPsetAndStudent[id(pset), id(student)] = grade
        return grade
    def pointsForPsetAndStudent(self, pset, student): # assume unique
        return self.gradeByPsetAndStudent[id(pset), id(student)]["points"]

In [49]:
r = example(IndexedLinkedRep1)
r.pointsForPsetAndStudent(r.psetWithNumber(1),
                          r.studentWithKerberos("al1ce"))

9.5

In [50]:
r.changeStudentName(r.studentWithKerberos("al1ce"), "Al-Ice")
r.pointsForPsetAndStudent(r.psetWithNumber(1),
                          r.studentWithKerberos("al1ce"))

9.5

### Grade List Queries

To support `gradesForStudent` and `gradesForPset` quickly, one natural approach is to store an index mapping each student to a list of grades, and similarly for psets:

In [51]:
class IndexedLinkedRep2(IndexedLinkedRep1):
    def __init__(self):
        super().__init__()
        self.gradesByStudent = {}
        self.gradesByPset = {}
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        self.gradesByStudent.setdefault(id(student), []).append(grade)
        self.gradesByPset.setdefault(id(pset), []).append(grade)
        return grade
    def gradesForStudent(self, student):
        return self.gradesByStudent[id(student)]
    def gradesForPset(self, pset):
        return self.gradesByPset[id(pset)]
    
class IndexedIDRep2(IndexedIDRep1):
    def __init__(self):
        super().__init__()
        self.gradesByStudent = {}
        self.gradesByPset = {}
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        self.gradesByStudent.setdefault(student["id"], []).append(grade)
        self.gradesByPset.setdefault(pset["id"], []).append(grade)
        return grade
    def gradesForStudent(self, student):
        return self.gradesByStudent[student["id"]]
    def gradesForPset(self, pset):
        return self.gradesByPset[pset["id"]]

In [52]:
for Rep in [LinkedRep, IndexedLinkedRep2, IDRep, IndexedIDRep2]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print(r.gradesForStudent(r.studentWithKerberos('b0b')))

# LinkedRep
[{'pset': {'number': 1, 'points': 10}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 6}, {'pset': {'number': 2, 'points': 15}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 12}]
# IndexedLinkedRep2
[{'pset': {'number': 1, 'points': 10}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 6}, {'pset': {'number': 2, 'points': 15}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 12}]
# IDRep
[{'id': 6, 'pset': 3, 'student': 2, 'points': 6}, {'id': 7, 'pset': 4, 'student': 2, 'points': 12}]
# IndexedIDRep2
[{'id': 6, 'pset': 3, 'student': 2, 'points': 6}, {'id': 7, 'pset': 4, 'student': 2, 'points': 12}]


But there are other natural choices, too.  We already have an object for each student and each pset, so we could cache (store) the list of grades for each *in the student/pset object itself*.

In [53]:
class NestedIndexedLinkedRep(IndexedLinkedRep1):
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        student["grades"] = []
        return student
    def addPset(self, number, points):
        pset = super().addPset(number, points)
        pset["grades"] = []
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        student["grades"].append(grade)
        pset["grades"].append(grade)
        return grade
    def gradesForStudent(self, student):
        return student["grades"]
    def gradesForPset(self, pset):
        return pset["grades"]
    
class NestedIndexedIDRep(IndexedIDRep1):
    def addStudent(self, name, kerberos): # same
        student = super().addStudent(name, kerberos)
        student["grades"] = []
        return student
    def addPset(self, number, points): # same
        pset = super().addPset(number, points)
        pset["grades"] = []
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        student["grades"].append(grade["id"])
        pset["grades"].append(grade["id"])
        return grade
    def gradesForStudent(self, student):
        return [self.gradeWithID(id) for id in student["grades"]]
    def gradesForPset(self, pset):
        return [self.gradeWithID(id) for id in pset["grades"]]

In [54]:
for Rep in [LinkedRep, NestedIndexedLinkedRep, IDRep, NestedIndexedIDRep]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print("BOB:", r.studentWithKerberos('b0b'))
    print("BOB's GRADES:", r.gradesForStudent(r.studentWithKerberos('b0b')))

# LinkedRep
BOB: {'name': 'Bob', 'kerberos': 'b0b'}
BOB's GRADES: [{'pset': {'number': 1, 'points': 10}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 6}, {'pset': {'number': 2, 'points': 15}, 'student': {'name': 'Bob', 'kerberos': 'b0b'}, 'points': 12}]
# NestedIndexedLinkedRep
BOB: {'name': 'Bob', 'kerberos': 'b0b', 'grades': [{'pset': {'number': 1, 'points': 10, 'grades': [{'pset': {...}, 'student': {'name': 'Alice', 'kerberos': 'al1ce', 'grades': [{...}]}, 'points': 9.5}, {...}]}, 'student': {...}, 'points': 6}, {'pset': {'number': 2, 'points': 15, 'grades': [{...}]}, 'student': {...}, 'points': 12}]}
BOB's GRADES: [{'pset': {'number': 1, 'points': 10, 'grades': [{'pset': {...}, 'student': {'name': 'Alice', 'kerberos': 'al1ce', 'grades': [{...}]}, 'points': 9.5}, {...}]}, 'student': {'name': 'Bob', 'kerberos': 'b0b', 'grades': [...]}, 'points': 6}, {'pset': {'number': 2, 'points': 15, 'grades': [{...}]}, 'student': {'name': 'Bob', 'kerberos': 'b0b', 'grades': [...]}, 'po

This is an example of a more deeply nested data structure.

There's no right or wrong choice here.  This last approach exposes some data structures to the user, which may or may not be useful.  It's probably good if you often need to access the grades from a pset or student.  On the other hand, the previous approach keeps a nice separation between indexes and data.

If we wanted to fully embrace nested data structures, we could also replace the `gradeByPsetAndStudent` by making the `grades` indexes into dictionaries instead of lists.  We can still loop over the contents using `.values()`, and now we can also look up a grade by both pset and student.

In [55]:
class AllNestedIndexedLinkedRep(IndexedLinkedRep):
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        student["grades"] = {}                          #NEW
        return student
    def addPset(self, number, points):
        pset = super().addPset(number, points)
        pset["grades"] = {}                             #NEW
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        student["grades"][id(pset)] = grade             #NEW
        pset["grades"][id(student)] = grade             #NEW
        return grade
    def pointsForPsetAndStudent(self, pset, student):
        return pset["grades"][id(student)]["points"]    #NEW
        #or: student["grades"][id(pset)]
    def gradesForStudent(self, student):
        return list(student["grades"].values())         #NEW
    def gradesForPset(self, pset):
        return list(pset["grades"].values())            #NEW
    
class AllNestedIndexedIDRep(IndexedIDRep):
    def addStudent(self, name, kerberos): # same
        student = super().addStudent(name, kerberos)
        student["grades"] = {}
        return student
    def addPset(self, number, points): # same
        pset = super().addPset(number, points)
        pset["grades"] = {}
        return pset
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        student["grades"][pset["id"]] = grade["id"]     #NEW
        pset["grades"][student["id"]] = grade["id"]     #NEW
        return grade
    def pointsForPsetAndStudent(self, pset, student):
        return self.gradeWithID(pset["grades"][student["id"]])["points"] #NEW
        #or: self.gradeWithID(student["grades"][pset["id"]])["points"]
    def gradesForStudent(self, student):
        return [self.gradeWithID(id)
                for id in student["grades"].values()]   #NEW
    def gradesForPset(self, pset):
        return [self.gradeWithID(id)
                for id in pset["grades"].values()]      #NEW

In [56]:
for Rep in [NestedIndexedLinkedRep, AllNestedIndexedLinkedRep, NestedIndexedIDRep, AllNestedIndexedIDRep]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print("BOB:", r.studentWithKerberos('b0b'))
    print("BOB's GRADES:", r.gradesForStudent(r.studentWithKerberos('b0b')))
    print("BOB's GRADE for PSET 1:", r.pointsForPsetAndStudent(
        r.psetWithNumber(1), r.studentWithKerberos('b0b')))

# NestedIndexedLinkedRep
BOB: {'name': 'Bob', 'kerberos': 'b0b', 'grades': [{'pset': {'number': 1, 'points': 10, 'grades': [{'pset': {...}, 'student': {'name': 'Alice', 'kerberos': 'al1ce', 'grades': [{...}]}, 'points': 9.5}, {...}]}, 'student': {...}, 'points': 6}, {'pset': {'number': 2, 'points': 15, 'grades': [{...}]}, 'student': {...}, 'points': 12}]}
BOB's GRADES: [{'pset': {'number': 1, 'points': 10, 'grades': [{'pset': {...}, 'student': {'name': 'Alice', 'kerberos': 'al1ce', 'grades': [{...}]}, 'points': 9.5}, {...}]}, 'student': {'name': 'Bob', 'kerberos': 'b0b', 'grades': [...]}, 'points': 6}, {'pset': {'number': 2, 'points': 15, 'grades': [{...}]}, 'student': {'name': 'Bob', 'kerberos': 'b0b', 'grades': [...]}, 'points': 12}]
BOB's GRADE for PSET 1: 6
# AllNestedIndexedLinkedRep
BOB: {'name': 'Bob', 'kerberos': 'b0b', 'grades': {1941170804440: {'pset': {'number': 1, 'points': 10, 'grades': {1941170804360: {'pset': {...}, 'student': {'name': 'Alice', 'kerberos': 'al1ce', 'grad

### Total Query

Finally, let's turn to `totalPointsForStudent`.  We already have a reasonable implementation that takes time linear in the number of grades for the student (using the indexes above).  But we can support these queries even faster by storing the answer, building it up as we add grades.  This is useful if we frequently query the same student.  Again, we could store it in a separate index or in the student objects.

In [57]:
class IndexedLinkedRep3(IndexedLinkedRep2):
    def __init__(self):
        super().__init__()
        self.totalPointsByStudent = {}
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        self.totalPointsByStudent[id(student)] = 0
        return student
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        self.totalPointsByStudent[id(student)] += points
        return grade
    def totalPointsForStudent(self, student):
        return self.totalPointsByStudent[id(student)]

class AllNestedIndexedLinkedRep3(AllNestedIndexedLinkedRep):
    def addStudent(self, name, kerberos):
        student = super().addStudent(name, kerberos)
        student["totalPoints"] = 0
        return student
    def addGrade(self, pset, student, points):
        grade = super().addGrade(pset, student, points)
        student["totalPoints"] += points
        return grade
    def totalPointsForStudent(self, student):
        return student["totalPoints"]

In [58]:
for Rep in [LinkedRep, IndexedLinkedRep3, AllNestedIndexedLinkedRep3]:
    print(f"# {Rep.__name__}")
    r = example(Rep)
    print("BOB:", r.studentWithKerberos('b0b'))
    print("BOB's TOTAL:", r.totalPointsForStudent(r.studentWithKerberos('b0b')))

# LinkedRep
BOB: {'name': 'Bob', 'kerberos': 'b0b'}
BOB's TOTAL: 18
# IndexedLinkedRep3
BOB: {'name': 'Bob', 'kerberos': 'b0b'}
BOB's TOTAL: 18
# AllNestedIndexedLinkedRep3
BOB: {'name': 'Bob', 'kerberos': 'b0b', 'grades': {1941170950696: {'pset': {'number': 1, 'points': 10, 'grades': {1941170808856: {'pset': {...}, 'student': {'name': 'Alice', 'kerberos': 'al1ce', 'grades': {1941170950696: {...}}, 'totalPoints': 9.5}, 'points': 9.5}, 1941170807416: {...}}}, 'student': {...}, 'points': 6}, 1941170804280: {'pset': {'number': 2, 'points': 15, 'grades': {1941170807416: {...}}}, 'student': {...}, 'points': 12}}, 'totalPoints': 18}
BOB's TOTAL: 18


## Another Example: Esports Data

Another more fun but more complex example of storing linked data is esports gameplay logs.  For most video games, we'd be interested in the following main types of objects:

* **Players**: tag, name, country, birthdate, etc.
* **Games**: which players played, which characters those players chose, which map they played on, who won / how many points each player obtained, game duration, when/where the game happened

Then there are many different queries we might want to support esports data analysis:

* How many games has this player won, and how many have they lost?
* What are the top 10 players in terms of total games won?
* What are the top 10 players in terms of win/loss ratio?
* What is the ELO-like ladder ranking of this player?  What are the top 10 players?
* Which character chosen by players has the highest win/loss ratio?
  For example, in Starcraft, is it statistically best to choose Terran, Zerg, or Protoss?
* Same questions above for each game map
* Etc.

A real-world example of such a database for Starcraft II is [Aligulac](http://aligulac.com/), which provides [a downloadable database and database spec](http://aligulac.com/about/db/).  (There's also an [official API](https://develop.battle.net/documentation/api-reference/starcraft-2-community-api) based on JSON input/output.)  Aligulac's database is designed for the relational database system PostgreSQL, so each object is flat (values are primitive values, not lists or dictionaries).  Note how every object has an ID and links (e.g. from a match game to the two players that played the game) are implemented by storing IDs.

The database dump even specifies a collection of indexes to build.  For example, each match game is indexed by ID, game version, the ID of a period object (what season the game was played in), the ID of the first player object, the ID of the second player object, the race of the first player, the race of the second player, etc.  These indexes enable quick filtering of the entire list of games to subsets of interest.  When we ask the Aligulac website for [the top player rankings in the current period/season](http://aligulac.com/periods/latest/), it doesn't sift through all game records and compute it — rather, it shows what it has stored in its data structure, and whenever new games get played, it updates that data structure, like we did above.

For example, here's [the current data](http://aligulac.com/periods/latest/) for which race won the most games in Starcraft II during the period from October 10, 2019 to October 19, 2019:

Race    |vs.| Race   | Wins | Losses | Win Ratio
--------|---|--------|------|--------|----------
Protoss |vs.| Terran | 133  | 132    | 50.19%
Protoss	|vs.| Zerg   | 215  | 272    | 44.15%
Terran  |vs.| Zerg   | 160  | 186    | 46.24%